# Modified Hamiltonian Path Puzzle

https://www.reddit.com/r/puzzles/comments/1kcgz1h/attempted_to_make_a_puzzle_trying_to_work_out_if/

Posted by u/LoafObread__ they mainly wanted to know if there was a unique solution. Other users quickly found alternate solutions.

This puzzle is effectively a modified/contrained Hamiltonian path problem. The constraints simply being the 2 required edges, and disallowed S pattern.

I've seen the Hamiltonian path problem formulated as an SMT problem solvable by z3, so this problem should be similarly easy to model. But will it be easy to solve? And what about finding all solutions?

- https://en.wikipedia.org/wiki/Hamiltonian_path
- https://en.wikipedia.org/wiki/Hamiltonian_path_problem

# Problem Setup

z3 actually has an official example showing conversion of a Hamiltonian path problem into z3 via the Python API. Perfect! Let's adapt the code for our purposes.

https://github.com/Z3Prover/z3/blob/master/examples/python/hamiltonian/hamiltonian.py

In [ ]:
import z3

In [ ]:
size = 9

Define edges of the complete lattice grid.

In [ ]:
edgelist = {
    (i,j): [
        (i+ip, j+jp) 
        for ip,jp in [(-1,0),(1,0),(0,-1),(0,1)] 
        if 0 <= i+ip < size and 0 <= j+jp < size
    ] 
    for i in range(size) 
        for j in range(size)
}
#edgelist

Remove the blocked edges from the graph.

In [ ]:
blocked = [((2,4), (3,4)), ((5,4), (6,4))]

for (k,kp) in blocked:
    edgelist[k].remove(kp)
    edgelist[kp].remove(k)

Define the solver and the variables: integers for each node representing its position on the Hamiltonian path.

In [ ]:
solver = z3.Solver()

cv = {(i,j): z3.Int(f'cv_{i}_{j}') for i,j in edgelist.keys()}

### Constraints

- Fix the start and end's value, then fill in the remaining nodes successor edges as the OR constrains.

In [ ]:
start = (0, size//2)
end = (size-1, size//2)

# fixed start and end
solver.add(cv[start]==0)
solver.add(cv[end] == size*size-1)

# continuous path
for k,v in edgelist.items():
    if k == end:
        continue
    solver.add(z3.Or([cv[kv]==cv[k]+1 for kv in v]))

- We have 2 edges that must be used, so add those mandatory constraints. Remember either direction is valid

In [ ]:
# required
required = [((4,0), (4,1)), ((4,7), (4,8))]

for (k,kp) in required:
    solver.add(z3.Or([cv[kp]==cv[k]+1, cv[k]==cv[kp]+1]))
    #print(z3.Or([cv[kp]==cv[k]+1, cv[k]==cv[kp]+1]))

- Add in the rotational symmetry constraint

In [ ]:
# rotational symmetry
for (i,j),v in edgelist.items():
    for (ip,jp) in v:
        #print((cv[(ip,jp)]==cv[(i,j)]+1) == (cv[(size-1-ip,size-1-jp)]==cv[(size-1-i,size-1-j)]+1))
        solver.add((cv[(ip,jp)]==cv[(i,j)]+1) == (cv[(size-1-ip,size-1-jp)]+1==cv[(size-1-i,size-1-j)]))

- Add the no 3x3 S shape constraint

In [ ]:
# no S
ips = [(1,-1), (1,0), (1,1), (0,1), (0,0), (0,-1), (-1,-1), (-1,0), (-1,1)]

def rot90(coords):
    return [(j, i) for i,j in coords]

def mirror(coords):
    return [(i, -j) for i,j in coords]

for (i,j) in edgelist.keys():
    if not all( 0<= i+ip < size and 0 <= j+jp < size for ip,jp in ips):
        continue
    for subpath_unt in [ips, mirror(ips), rot90(ips), mirror(rot90(ips))]:
        for subpath in [subpath_unt, subpath_unt[::-1]]:
            #print(z3.Or([
            #    cv[(i+ipp, j+jpp)] != cv[(i+ip, j+jp)]+1 
            #    for (ip,jp),(ipp,jpp) in zip(subpath[:-1],subpath[1:])]
            #))
            solver.add(z3.Or([
                cv[(i+ipp, j+jpp)] != cv[(i+ip, j+jp)]+1 
                for (ip,jp),(ipp,jpp) in zip(subpath[:-1],subpath[1:])]
            ))

- Add no 4 in a row constraint

In [ ]:
ips = [(0, jp) for jp in range(4)]

for (i,j) in edgelist.keys():
    for subpath_unt in [ips, mirror(ips), rot90(ips), rot90(mirror(ips))]:
        for subpath in [subpath_unt, subpath_unt[::-1]]:
            if not all( 0<= i+ip < size and 0 <= j+jp < size for ip,jp in subpath):
                continue
            #print(z3.Or([
            #    cv[(i+ipp, j+jpp)] != cv[(i+ip, j+jp)]+1 
            #    for (ip,jp),(ipp,jpp) in zip(subpath[:-1],subpath[1:])]
            #))
            solver.add(z3.Or([
                cv[(i+ipp, j+jpp)] != cv[(i+ip, j+jp)]+1 
                for (ip,jp),(ipp,jpp) in zip(subpath[:-1],subpath[1:])]
            ))

## Solving

Perform the solve

In [ ]:
print(solver.check())

In [ ]:
#print(solver.model())

Extract the path from the solution

In [ ]:
def extract_path(model):
    path = [None] * (size * size)
    for k,var in cv.items():
        path[model[var].as_long()] = k
    return path

In [ ]:
model = solver.model()
path = extract_path(model)
path

### Find all solutions

In [ ]:
solutions = []
while solver.check() == z3.sat:
    model = solver.model()
    solutions.append(model)
    print(f"Solution {len(solutions)}")
    solver.add(z3.Or([var != model[var] for var in cv.values()]))

In [ ]:
len(solutions)

## Visualization

In [ ]:
import networkx as nx

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Construct the graph
G = nx.Graph()
G.add_nodes_from(edgelist.keys())

#for k,v in edgelist.items():
#    for kv in v:
#        G.add_edge(k, kv)

for k,kp in zip(path[:-1], path[1:]):
    G.add_edge(k, kp)

# positions 
pos_dict = {k: tuple(reversed(k)) for k in edgelist.keys()}

In [ ]:
# Draw the graph
#nx.draw(G, pos=pos_dict, with_labels=True, node_size=500, node_color='lightblue', font_size=10, font_weight='bold')

nx.draw(G, pos=pos_dict, node_size=50, node_color='black', edge_color='blue')

for ((i,j),(ip,jp)) in blocked:
    plt.plot(
        [(j+jp-(j==jp))/2, (j+jp+(j==jp))/2],
        [(i+ip-(i==ip))/2, (i+ip+(i==ip))/2], 
    c='r')
    
plt.gca().set_aspect('equal')
plt.show()

### Graph all solutions

In [ ]:
for i_sol,model in enumerate(solutions):
    path = extract_path(model)
    #print(path)
    
    G = nx.Graph()
    G.add_nodes_from(edgelist.keys())

    for k,kp in zip(path[:-1], path[1:]):
        G.add_edge(k, kp)

    pos_dict = {k: tuple(reversed(k)) for k in edgelist.keys()}
    
    plt.figure()
    
    nx.draw(G, pos=pos_dict, node_size=50, node_color='black', edge_color='blue')

    for ((i,j),(ip,jp)) in blocked:
        plt.plot(
            [(j+jp-(j==jp))/2, (j+jp+(j==jp))/2],
            [(i+ip-(i==ip))/2, (i+ip+(i==ip))/2], 
        c='r')

    plt.gca().set_aspect('equal')
    plt.savefig(f'ModifiedHamiltonanPath_Solution_{i_sol+1}.png')
plt.show()